In [ ]:
!pip install datasets
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.7 MB/s eta 0:00:00


## Setting up WandB:

In [ ]:
import wandb

wandb.login()

# HYPERPAREMETERS
LEARNING_RATE = 5e-5
EPOCHS = 30
BATCH_SIZE = 8
run = wandb.init(
    # Set the project where this run will be logged
    project="Data Security Project", name= "Distill Bert new dataset, First legit training attempt",
    # Track hyperparameters and run metadata
    config={
        "learning_rate": 0.01,
        "Batch_size": 8,
        "epochs": 15,
    },
)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: anasnamouchi. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


## Deserializing and checking PKL file:

In [ ]:
import pickle
with open("agg_data.pkl",'rb') as dataframe_file:
  opened_dataframe = pickle.load(dataframe_file)

opened_dataframe.head()

,label,segment
0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0]",Privacy Policy Sci-News.com is committed to pr...
1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",Information that Sci-News.com May Collect Onli...
2,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","- if you contact us, we may keep a record of t..."
3,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",- details of your visits to our site including...
4,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]",Sci-News.com does not knowingly collect or sol...


## Converting int to float in the dataset (required by model):

In [ ]:
opened_dataframe['label'] = opened_dataframe['label'].apply(lambda x: [float(i) for i in x])


# Check type of labels
type(opened_dataframe['label'][0][0])
print(type(opened_dataframe['label'][0]))
print(opened_dataframe['label'][0])

<class 'list'>
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0]


## Load other libraries and split dataset:

In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from sklearn.model_selection import train_test_split
from datasets import Dataset
import evaluate
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, hamming_loss



train_df, test_df = train_test_split(opened_dataframe, test_size=0.3)


## Convert to Huggingface Dataset:

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

## Tokenize Data:

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenize the text data
def tokenize_function(examples):
    return tokenizer(examples['segment'], padding="max_length", truncation=True)

train_dataset_tokenized = train_dataset.map(tokenize_function, batched=True)
test_dataset_tokenized = test_dataset.map(tokenize_function, batched=True)

# Load DistilBERT model for sequence classification with 3 labels (multi-label classification)
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased",
                                                            num_labels=12,
                                                            problem_type="multi_label_classification",
                                                            )

# Data Collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/2651 [00:00<?, ? examples/s]

Map:   0%|          | 0/1137 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Take a smaller subset of the data for testing:

In [ ]:
# Optional: Take smaller subsets for quick testing
small_train_dataset = train_dataset_tokenized.shuffle(seed=42).select(range(1000))
small_eval_dataset = test_dataset_tokenized.shuffle(seed=42).select(range(200))

['label', 'segment', '__index_level_0__', 'input_ids', 'attention_mask']
['label', 'segment', '__index_level_0__', 'input_ids', 'attention_mask']
[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0

## Testing: One single run through model:

In [ ]:
# Assume 'input_ids' and 'attention_mask' are already tokenized and prepared for one example
input_ids = torch.tensor(small_train_dataset['input_ids'][0:1])  # Select the first example
attention_mask = torch.tensor(small_train_dataset['attention_mask'][0:1])  # Select the first example

# move input tensors to the same device as the model
device = next(model.parameters()).device  # Get model's device
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)

# Pass the inputs through the model
outputs = model(input_ids=input_ids, attention_mask=attention_mask)

# Get the logits
logits = outputs.logits

# Print the shape of the logits
print(logits.shape)

print(torch.tensor(small_eval_dataset['label']).shape)

# => size for one example is ok
print(type(small_eval_dataset['label']))
print(type(small_eval_dataset['label'][0]))
print(type(small_eval_dataset['label'][0][0]))
print(small_eval_dataset['label'][0][0])

torch.Size([1, 12])
torch.Size([200, 12])
<class 'list'>
<class 'list'>
<class 'float'>
0.0


In [ ]:
# Define training arguments

training_args = TrainingArguments(
    output_dir="30_epochs_model",
    eval_strategy="epoch",               # Evaluate after every epoch
    report_to="wandb",
    num_train_epochs= EPOCHS,                 # Total number of epochs
    save_strategy="epoch",               # Save the best model at the end of each epoch
    load_best_model_at_end=True,         # Load the best model based on validation loss
    learning_rate= 5e-5,                  # Learning rate for the optimizer
    weight_decay=0.01,                   # Weight decay for regularization
    per_device_train_batch_size= 8,      # Training batch size
    per_device_eval_batch_size= 8,       # Evaluation batch size
    gradient_accumulation_steps=4,       # Accumulate gradients for fewer backward passes
    logging_strategy="epoch",            # Log metrics at intervals of steps
    log_level="info",                    # Log level (e.g., "info" or "error")
    log_level_replica="warning",        # Adjust logs for distributed training replicas
    logging_dir="./logs",               # Directory for storing logs
    logging_steps = 100,
    metric_for_best_model="eval_loss",  # Metric to track the best model
    fp16=True,  # Enable mixed precision

    )




'''def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Apply sigmoid to convert logits to probabilities
    probs = torch.sigmoid(torch.tensor(logits)).numpy()

    # Use threshold of 0.5 to convert probabilities to binary predictions
    predictions = (probs > 0.5).astype(int)

    # Compute accuracy score by comparing each label prediction with the true label
    accuracy = accuracy_score(labels, predictions)

    # Calculate Precision, Recall, F1-score for each label
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='macro', zero_division=0
    )


    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }
'''


def compute_metrics(eval_pred):
    """
    Compute metrics for multilabel classification.
    :param eval_pred: Tuple (predictions, labels)
    :return: Dictionary with metric values
    """
    logits, labels = eval_pred
    # Apply sigmoid to logits for multilabel classification
    probs = 1 / (1 + np.exp(-logits))
    # Convert probabilities to binary predictions (0 or 1)
    preds = (probs > 0.5).astype(int)

    # Exact match ratio: Proportion of samples with all labels correct
    exact_match = np.all(preds == labels, axis=1).mean()

    # Multilabel accuracy: Average accuracy across all labels
    multilabel_accuracy = (preds == labels).mean()

    # F1 Score (macro and micro)
    f1_macro = f1_score(labels, preds, average="macro")
    f1_micro = f1_score(labels, preds, average="micro")

    # Hamming loss
    hamming = hamming_loss(labels, preds)

    return {
        "exact_match": exact_match,
        "multilabel_accuracy": multilabel_accuracy,
        "f1_macro": f1_macro,
        "f1_micro": f1_micro,
        "hamming_loss": hamming,
    }


PyTorch: setting up devices


In [ ]:
# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_tokenized,
    eval_dataset=test_dataset_tokenized,
    compute_metrics=compute_metrics,
    data_collator=data_collator,


)

# Fine-tune the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)

# Save the model
model.save_pretrained("./distilbert-multi-label")

Using auto half precision backend
The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: segment, __index_level_0__. If segment, __index_level_0__ are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 2,651
  Num Epochs = 30
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 4
  Total optimization steps = 2,490
  Number of trainable parameters = 66,962,700
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Exact Match,Multilabel Accuracy,F1 Macro,F1 Micro,Hamming Loss,Runtime,Samples Per Second,Steps Per Second
1,0.320500,0.183989,0.492524,0.936236,0.696602,0.760989,0.063764,5.073500,224.104000,28.185000
2,0.213800,0.191333,0.481970,0.935649,0.729234,0.761283,0.064351,4.898800,232.098000,29.191000
3,0.151900,0.215738,0.490765,0.934403,0.702549,0.753783,0.065597,4.885100,232.749000,29.273000
4,0.107500,0.222007,0.497801,0.936382,0.731127,0.769026,0.063618,5.071200,224.205000,28.198000
5,0.081500,0.248133,0.466139,0.931985,0.736711,0.758961,0.068015,4.785700,237.585000,29.881000
6,0.081100,0.250495,0.483729,0.935210,0.731774,0.767735,0.064790,4.974200,228.580000,28.748000
7,0.066100,0.259146,0.496042,0.937995,0.744727,0.775000,0.062005,4.870200,233.463000,29.363000
8,0.060500,0.268963,0.469657,0.934770,0.734073,0.767866,0.065230,4.806900,236.537000,29.749000
9,0.045000,0.280388,0.495163,0.936895,0.738883,0.769354,0.063105,4.756400,239.049000,30.065000
10,0.038000,0.284831,0.478452,0.935356,0.737193,0.766172,0.064644,4.729600,240.403000,30.235000


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: segment, __index_level_0__. If segment, __index_level_0__ are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 1137
  Batch size = 8
Saving model checkpoint to 30_epochs_model/checkpoint-83
Configuration saved in 30_epochs_model/checkpoint-83/config.json
Model weights saved in 30_epochs_model/checkpoint-83/model.safetensors
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: segment, __index_level_0__. If segment, __index_level_0__ are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 1137
  Batch size = 8
Saving model checkpoint to 30_epochs_

Configuration saved in ./distilbert-multi-label/config.json


{'eval_loss': 0.18398940563201904, 'eval_exact_match': 0.4925241864555849, 'eval_multilabel_accuracy': 0.9362357080035181, 'eval_f1_macro': 0.6966024223303714, 'eval_f1_micro': 0.760989010989011, 'eval_hamming_loss': 0.06376429199648197, 'eval_runtime': 4.8672, 'eval_samples_per_second': 233.603, 'eval_steps_per_second': 29.38, 'epoch': 30.0}


Model weights saved in ./distilbert-multi-label/model.safetensors


In [ ]:
# save folder from model locally, transform it to zip
!zip -r 30_epochs_model/.zip 30_epochs_model/


  adding: 30_epochs_model/ (stored 0%)
  adding: 30_epochs_model/checkpoint-2324/ (stored 0%)
  adding: 30_epochs_model/checkpoint-2324/rng_state.pth (deflated 25%)
  adding: 30_epochs_model/checkpoint-2324/scheduler.pt (deflated 56%)
  adding: 30_epochs_model/checkpoint-2324/training_args.bin (deflated 51%)
  adding: 30_epochs_model/checkpoint-2324/optimizer.pt (deflated 35%)
  adding: 30_epochs_model/checkpoint-2324/model.safetensors (deflated 8%)
  adding: 30_epochs_model/checkpoint-2324/trainer_state.json (deflated 80%)
  adding: 30_epochs_model/checkpoint-2324/config.json (deflated 57%)
  adding: 30_epochs_model/checkpoint-1245/ (stored 0%)
  adding: 30_epochs_model/checkpoint-1245/rng_state.pth (deflated 25%)
  adding: 30_epochs_model/checkpoint-1245/scheduler.pt (deflated 56%)
  adding: 30_epochs_model/checkpoint-1245/training_args.bin (deflated 51%)
  adding: 30_epochs_model/checkpoint-1245/optimizer.pt (deflated 35%)
  adding: 30_epochs_model/checkpoint-1245/model.safetensors 